In [ ]:
import os
os.chdir('../')

from medclip import MedCLIPModel, MedCLIPVisionModelViT
from medclip.modeling_medclip import MedCLIPVisionModel
from medclip import MedCLIPProcessor

from PIL import Image

In [ ]:
# prepare for the demo image and texts
processor = MedCLIPProcessor()

image = Image.open('./example_data/view1_frontal.jpg')
inputs = processor(
    text=["lungs remain severely hyperinflated with upper lobe emphysema", 
        "opacity left costophrenic angle is new since prior exam ___ represent some loculated fluid cavitation unlikely"], 
    images=image, 
    return_tensors="pt", 
    padding=True
    )

In [ ]:
# init model from pretrained checkpoint
model = MedCLIPModel(vision_cls=MedCLIPVisionModelViT)
model.cuda()

In [ ]:
outputs = model(**inputs)

In [ ]:
outputs.keys()

In [ ]:
outputs['logits'].shape

In [ ]:
outputs['logits']

In [ ]:
from data_utils import load_dataset, LESION_TYPE

In [ ]:
medclip = MedCLIPModel(vision_cls=MedCLIPVisionModelViT)

def clip_preprocess(image):
    return processor(images=image, return_tensors="pt", padding=True)

ham_train, ham_test = load_dataset("HAM10000", transform=clip_preprocess)
print(f"Train size: {len(ham_train)}")
print(f"Test size: {len(ham_test)}")
print(ham_train)
print(ham_test)

BATCH_SIZE = 128



# Implement a zero-shot function for medclip

In [ ]:
# implement a zero-shot function for medclip

import torch
import torchvision
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
from build.lib.medclip.constants import BERT_TYPE, IMG_MEAN, IMG_STD, IMG_SIZE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from data_utils import load_ham10000_dataset, LESION_TYPE

BATCH_SIZE = 128

def medclip_zero_shot_inline(test_dataset, classes, batch_size=BATCH_SIZE):
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Data loader for the dataset
    data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    print(f"Device: {device}")

    # Initialize MedClip Models
    model = MedCLIPModel(vision_cls=MedCLIPVisionModelViT).to(device)

    # Prepare text prompts
    text_prompts = [f"a photo of a {c}, a type of skin lesion." for c in classes]
    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(BERT_TYPE)

    # Tokenize text prompts and convert to tensors
    text_tokens = [tokenizer(text, return_tensors='pt', padding=True, truncation=True, add_special_tokens=True) for text in text_prompts]

    # Encode text prompts using MedClip's text model
    # Inside the medclip_zero_shot function
    text_features = [
        model.encode_text(
            input_ids=tokens['input_ids'].to(device), 
            attention_mask=tokens['attention_mask'].to(device)
        ) 
        for tokens in text_tokens
    ]

    # Initialize variables for accuracy calculation
    correct = 0
    total = 0

    for images, labels in tqdm(data_loader):
        images, labels = images.to(device), labels.to(device)

        # TODO: Encode images using MedClip's vision model
        image_features = model.encode_image(images)

        # Flatten text_features into a single 2D tensor
        text_features_tensor = torch.cat(text_features, dim=0)

        # Calculate similarity and make predictions
        similarity = torch.matmul(image_features, text_features_tensor.t())
        _, predictions = similarity.max(dim=-1)

        # Update correct and total counts
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

    return correct / total

# Load HAM10000 dataset
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((IMG_SIZE, IMG_SIZE)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[IMG_MEAN], std=[IMG_STD])
])

train_dataset, test_dataset = load_ham10000_dataset(data_dir="data/ham10000/", transform=transform)
classes = list(LESION_TYPE.values())  # From the data_utils.py file

# Run zero-shot classification
acc = medclip_zero_shot_inline(test_dataset, classes)
print(f"Accuracy: {acc:.2f}")



In [ ]:
def medclip_zero_shot(model, test_dataset, classes, batch_size=BATCH_SIZE):
    # Data loader for the dataset
    data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    # Prepare text prompts
    text_prompts = [f"a photo of a {c}, a type of skin lesion." for c in classes]
    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(BERT_TYPE)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

    # Tokenize text prompts and convert to tensors
    text_tokens = [tokenizer(text, return_tensors='pt', padding=True, truncation=False, add_special_tokens=True) for text in text_prompts]

    # Encode text prompts using MedClip's text model
    # Inside the medclip_zero_shot function
    text_features = [
        model.encode_text(
            input_ids=tokens['input_ids'].to(device), 
            attention_mask=tokens['attention_mask'].to(device)
        ) 
        for tokens in text_tokens
    ]

    # Initialize variables for accuracy calculation
    correct = 0
    total = 0

    for images, labels in tqdm(data_loader):
        images, labels = images.to(device), labels.to(device)
        # Encode images using MedClip's vision model
        # with torch.no_grad():
        image_features = model.encode_image(images)
        # Flatten text_features into a single 2D tensor
        text_features_tensor = torch.cat(text_features, dim=0)

        # Calculate similarity and make predictions
        similarity = torch.matmul(image_features, text_features_tensor.t())
        _, predictions = similarity.max(dim=-1)

        # Update correct and total counts
        correct += (predictions == labels).sum().item()
        total += len(labels)

    return correct / total

## Load HAM10000 dataset and test MedClip's zero-shot capabilities

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((IMG_SIZE, IMG_SIZE)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[IMG_MEAN], std=[IMG_STD])
])

ham_train, ham_test = load_ham10000_dataset(data_dir="data/ham10000/", transform=transform)
classes = list(LESION_TYPE.values())  # From the data_utils.py file


MedCLIP_ResNet50_model

In [ ]:
# load MedCLIP-ResNet50
MedCLIP_ResNet50_model = MedCLIPModel(vision_cls=MedCLIPVisionModel).to(device)
accuracy = medclip_zero_shot(MedCLIP_ResNet50_model, ham_train, classes)
print(f"\nAccuracy = {100*accuracy:.3f}%")

MedCLIP_ViT_model

In [ ]:
# load MedCLIP-ViT
MedCLIP_ViT_model = MedCLIPModel(vision_cls=MedCLIPVisionModelViT).to(device)
accuracy = medclip_zero_shot(MedCLIP_ViT_model, ham_train, classes)
print(f"\nAccuracy = {100*accuracy:.3f}%")

## Load NIH Chest X-ray dataset

In [1]:
from data_utils import load_nih_dataset_split, NIH_CLASS_TYPES
import torch
from medclip import MedCLIPModel, MedCLIPVisionModelViT, MedCLIPVisionModel
from build.lib.medclip.constants import BERT_TYPE, IMG_MEAN, IMG_STD, IMG_SIZE
import torchvision
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn.functional as F

from PIL import Image

import os
os.chdir('../')

BATCH_SIZE = 128

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((IMG_SIZE, IMG_SIZE)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[IMG_MEAN], std=[IMG_STD])
])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# NIH_CLASS_TYPES
classes = list(NIH_CLASS_TYPES)  # From the data_utils.py file
classes
nih_train, nih_test = load_nih_dataset_split(transform=transform)
nih_train, nih_test

Selecting random image directory...
Loading csv file from data/nih/Data_Entry_2017.csv
Loading dataset from data/nih/images
Loading NIH dataset... data/nih/Data_Entry_2017.csv
Loading images from... data/nih/images
Original Dataset length: 112120
Train dataset length: 89696
Test dataset length: 22424


(<torch.utils.data.dataset.Subset at 0x187c1a9b350>,
 <torch.utils.data.dataset.Subset at 0x187c1a9b2d0>)

In [2]:
def medclip_zero_shot(model, test_dataset, classes, batch_size=BATCH_SIZE):
    # Data loader for the dataset
    data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    # Prepare text prompts
    text_prompts = [f"a photo of a {c}, a type of skin lesion." for c in classes]
    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(BERT_TYPE)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

    # Tokenize text prompts and convert to tensors
    text_tokens = [tokenizer(text, return_tensors='pt', padding=True, truncation=False, add_special_tokens=True) for text in text_prompts]

    # Encode text prompts using MedClip's text model
    # Inside the medclip_zero_shot function
    text_features = [
        model.encode_text(
            input_ids=tokens['input_ids'].to(device), 
            attention_mask=tokens['attention_mask'].to(device)
        ) 
        for tokens in text_tokens
    ]

    # Initialize variables for accuracy calculation
    correct = 0
    total = 0

    for images, labels in tqdm(data_loader):
        images, labels = images.to(device), labels.to(device)
        image_features = model.encode_image(images)
        text_features_tensor = torch.cat(text_features, dim=0)

        similarity = torch.matmul(image_features, text_features_tensor.t())
        probabilities = F.sigmoid(similarity)  # Convert to probabilities
        predictions = (probabilities > 0.5).float()  # Apply threshold to get binary predictions

        # Update correct and total counts
        # Calculate correct predictions in a multi-label scenario
        correct_preds = (predictions == labels).all(dim=1).sum().item()
        correct += correct_preds
        total += len(labels)

    return correct / total

In [3]:
# load MedCLIP-ResNet50
MedCLIP_ResNet50_model = MedCLIPModel(vision_cls=MedCLIPVisionModel).to(device)
MedCLIP_ResNet50_model
accuracy = medclip_zero_shot(MedCLIP_ResNet50_model, nih_train, classes)
print(f"\nAccuracy = {100*accuracy:.3f}%")

c:\Users\mario\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mario\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.

Device: cuda


100%|██████████| 701/701 [13:59<00:00,  1.20s/it] 


Accuracy = 0.852%


In [4]:
# load MedCLIP-ViT
MedCLIP_ViT_model = MedCLIPModel(vision_cls=MedCLIPVisionModelViT).to(device)
accuracy = medclip_zero_shot(MedCLIP_ViT_model, nih_train, classes)
print(f"\nAccuracy = {100*accuracy:.3f}%")

c:\Users\mario\anaconda3\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at

Device: cuda


100%|██████████| 701/701 [1:01:22<00:00,  5.25s/it]  


Accuracy = 0.618%
